# python数据分析之Pandas-3

**导入包**

In [1]:
import pandas as pd
import numpy as np

**读入数据**

In [2]:
# df = pd.read_excel(r'C:\Users\Lovetianyi\Desktop\python\作业4\movie_data2.xlsx', index_col = 0)
df = pd.read_excel('../lec4/movie_data2.xlsx', index_col = 0)


In [3]:
df[:5]

,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点
0,控方证人,42995,剧情/悬疑/犯罪,美国,1957-12-17 00:00:00,116,1957,9.5,美国
1,美丽人生,327855,剧情/喜剧/爱情,意大利,1997-12-20 00:00:00,116,1997,9.5,意大利
2,阿甘正传,580897,剧情/爱情,美国,1994-06-23 00:00:00,142,1994,9.4,洛杉矶首映
3,霸王别姬,478523,剧情/爱情/同性,中国大陆,1993-01-01 00:00:00,171,1993,9.4,香港
4,泰坦尼克号,157074,剧情/爱情/灾难,美国,2012-04-10 00:00:00,194,2012,9.4,中国大陆


## 3.1 数据重塑和轴向旋转 

### (1)层次化索引 

层次化索引是pandas的一项重要功能，它能使我们在一个轴上拥有多个索引。

### Series的层次化索引： 

In [14]:
s = pd.Series(np.arange(1, 10), index=[
              ['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 2, 3, 1, 2, 3]])
s2 = pd.Series(np.arange(1, 10), index=[
              ['a']*3+['b']*3+['c']*3, [1, 2, 3]*3])
print(s,'\n') #类似于合并单元格
print(s2,'\n') 

a  1    1
   2    2
   3    3
b  1    4
   2    5
c  3    6
   1    7
d  2    8
   3    9
dtype: int64 

a  1    1
   2    2
   3    3
b  1    4
   2    5
   3    6
c  1    7
   2    8
   3    9
dtype: int64 



In [15]:
s.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('c', 3),
            ('c', 1),
            ('d', 2),
            ('d', 3)],
           )

In [16]:
s['a'] #外层索引

1    1
2    2
3    3
dtype: int64

In [19]:
s[1]

2

In [7]:
s['a':'c'] #切片

a  1    1
   2    2
   3    3
b  1    4
   2    5
c  3    6
   1    7
dtype: int32

In [8]:
s[:,1] #内层索引

a    1
b    4
c    7
dtype: int32

In [9]:
s['c',3] #提取具体的值

6

**通过unstack方法可以将Series变成一个DataFrame**
![pic](p1.jpg)

In [20]:
s.unstack()

,1,2,3
a,1.0,2.0,3.0
b,4.0,5.0,NaN
c,7.0,NaN,6.0
d,NaN,8.0,9.0


In [21]:
s.unstack().stack() #形式上的相互转换

a  1    1.0
   2    2.0
   3    3.0
b  1    4.0
   2    5.0
c  1    7.0
   3    6.0
d  2    8.0
   3    9.0
dtype: float64

### Dataframe的层次化索引： 

对于DataFrame来说，行和列都能进行层次化索引。

In [32]:
data_frame = pd.DataFrame(np.arange(12).reshape(4,3))
data_frame

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11


In [33]:
data = pd.DataFrame(np.arange(12).reshape(4,3), index = [['a','a','b','b'],[1,2]*2], columns = [['A','A','B'],['A1','A2','B1']])
data

A       B
    A1  A2  B1
a 1  0   1   2
  2  3   4   5
b 1  6   7   8
  2  9  10  11

In [28]:
data['A']

A1  A2
a 1   0   1
  2   3   4
b 1   6   7
  2   9  10

In [36]:
data.loc['a']

A     B
  A1 A2 B1
1  0  1  2
2  3  4  5

In [37]:
data.loc['a':'b']

A       B
    A1  A2  B1
a 1  0   1   2
  2  3   4   5
b 1  6   7   8
  2  9  10  11

In [38]:
data.loc['a']['A']

,A1,A2
1,0,1
2,3,4


In [29]:
data['B']

B1
a 1   2
  2   5
b 1   8
  2  11

In [46]:
data.index.names = ["row1","row2"]
data.columns.names = ["col1", "col2"]
data

col1       A       B
col2      A1  A2  B1
row1 row2           
a    1     0   1   2
     2     3   4   5
b    1     6   7   8
     2     9  10  11

In [47]:
data.swaplevel("row1","row2") #位置调整

col1       A       B
col2      A1  A2  B1
row2 row1           
1    a     0   1   2
2    a     3   4   5
1    b     6   7   8
2    b     9  10  11

### 了解了层次化索引的基本知识之后，我们试着将电影数据也处理成一种多层索引的结构。 

In [48]:
df.index #默认索引

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            38723, 38724, 38725, 38726, 38727, 38728, 38729, 38730, 38731,
            38732],
           dtype='int64', length=38733)

### 把产地和年代同时设成索引，产地是外层索引，年代为内层索引。 

#### set_index可以把列变成索引

#### reset_index是把索引变成列 

In [49]:
df = df.set_index(["产地", "年代"])
df

名字    投票人数  \
产地   年代                                                                
美国   1957                                               控方证人   42995   
意大利  1997                                              美丽人生   327855   
美国   1994                                               阿甘正传  580897   
中国大陆 1993                                               霸王别姬  478523   
美国   2012                                             泰坦尼克号   157074   
...                                                      ...     ...   
中国大陆 1986                                               血溅画屏      95   
     1986                                             魔窟中的幻想      51   
俄罗斯  1977  列宁格勒围困之星火战役 Блокада: Фильм 2: Ленинградский ме...      32   
美国   2018                                             复仇者联盟3  123456   
     2018                                             复仇者联盟3  123456   

                       类型                 上映时间   时长        评分   首映地点  
产地   年代                                                               
美国   1957        剧情/悬疑/犯罪  1957-12-17 00:00:00  116  9.500000     美国  
意大利  1997        剧情/喜剧/爱情  1997-12-20 00:00:00  116  9.500000    意大利  
美国   1994           剧情/爱情  1994-06-23 00:00:00  142  9.400000  洛杉矶首映  
中国大陆 1993        剧情/爱情/同性  1993-01-01 00:00:00  171  9.400000     香港  
美国   2012        剧情/爱情/灾难  2012-04-10 00:00:00  194  9.400000   中国大陆  
...                   ...                  ...  ...       ...    ...  
中国大陆 1986  剧情/悬疑/犯罪/武侠/古装  1905-06-08 00:00:00   91  7.100000     美国  
     1986        惊悚/恐怖/儿童  1905-06-08 00:00:00   78  8.000000     美国  
俄罗斯  1977           剧情/战争  1905-05-30 00:00:00   97  6.600000     美国  
美国   2018           剧情/科幻  2018-05-04 00:00:00  142  6.935635     美国  
     2018           剧情/科幻  2018-05-04 00:00:00  142  6.935635     美国  

[38733 rows x 7 columns]

### 每一个索引都是一个元组 

In [18]:
df.index[0]

('美国', 1994)

#### 获取所有的美国电影，由于产地信息已经变成了索引，因此要是用.loc方法。 

In [19]:
df.loc["美国"] #行标签索引行数据，注意索引多行时两边都是闭区间

,名字,投票人数,类型,上映时间,时长,评分,首映地点
年代,,,,,,,
1994,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.600000,多伦多电影节
1957,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.500000,美国
1994,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.400000,洛杉矶首映
2012,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.400000,中国大陆
1993,辛德勒的名单,306904,剧情/历史/战争,1993-11-30 00:00:00,195,9.400000,华盛顿首映
...,...,...,...,...,...,...,...
1987,零下的激情,199,剧情/爱情/犯罪,1987-11-06 00:00:00,98,7.400000,美国
1986,离别秋波,240,剧情/爱情/音乐,1986-02-19 00:00:00,90,8.200000,美国
1986,极乐森林,45,纪录片,1986-09-14 00:00:00,90,8.100000,美国


In [20]:
df.loc["中国大陆"]

,名字,投票人数,类型,上映时间,时长,评分,首映地点
年代,,,,,,,
1993,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港
1961,大闹天宫,74881,动画/奇幻,1905-05-14 00:00:00,114,9.2,上集
2015,穹顶之下,51113,纪录片,2015-02-28 00:00:00,104,9.2,中国大陆
1982,茶馆,10678,剧情/历史,1905-06-04 00:00:00,118,9.2,美国
1988,山水情,10781,动画/短片,1905-06-10 00:00:00,19,9.2,美国
...,...,...,...,...,...,...,...
1986,T省的八四、八五,380,剧情,1905-06-08 00:00:00,94,8.7,美国
1986,失踪的女中学生,101,儿童,1905-06-08 00:00:00,102,7.4,美国
1986,血战台儿庄,2908,战争,1905-06-08 00:00:00,120,8.1,美国


#### 这样做的最大好处是我们可以简化很多的筛选环节

#### 每一个索引是一个元组 

In [21]:
df = df.swaplevel("产地", "年代") #调换标签顺序
df

名字    投票人数  \
年代   产地                                                                
1994 美国                                               肖申克的救赎  692795   
1957 美国                                                 控方证人   42995   
1997 意大利                                               美丽人生   327855   
1994 美国                                                 阿甘正传  580897   
1993 中国大陆                                               霸王别姬  478523   
...                                                      ...     ...   
1935 美国                                                1935年      57   
1986 中国大陆                                               血溅画屏      95   
     中国大陆                                             魔窟中的幻想      51   
1977 俄罗斯   列宁格勒围困之星火战役 Блокада: Фильм 2: Ленинградский ме...      32   
2018 美国                                               复仇者联盟3  123456   

                       类型                 上映时间   时长        评分    首映地点  
年代   产地                                                                
1994 美国             剧情/犯罪  1994-09-10 00:00:00  142  9.600000  多伦多电影节  
1957 美国          剧情/悬疑/犯罪  1957-12-17 00:00:00  116  9.500000      美国  
1997 意大利         剧情/喜剧/爱情  1997-12-20 00:00:00  116  9.500000     意大利  
1994 美国             剧情/爱情  1994-06-23 00:00:00  142  9.400000   洛杉矶首映  
1993 中国大陆        剧情/爱情/同性  1993-01-01 00:00:00  171  9.400000      香港  
...                   ...                  ...  ...       ...     ...  
1935 美国             喜剧/歌舞  1935-03-15 00:00:00   98  7.600000      美国  
1986 中国大陆  剧情/悬疑/犯罪/武侠/古装  1905-06-08 00:00:00   91  7.100000      美国  
     中国大陆        惊悚/恐怖/儿童  1905-06-08 00:00:00   78  8.000000      美国  
1977 俄罗斯            剧情/战争  1905-05-30 00:00:00   97  6.600000      美国  
2018 美国             剧情/科幻  2018-05-04 00:00:00  142  6.935704      美国  

[38163 rows x 7 columns]

In [22]:
df.loc[1994]

,名字,投票人数,类型,上映时间,时长,评分,首映地点
产地,,,,,,,
美国,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节
美国,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
法国,这个杀手不太冷,662552,剧情/动作/犯罪,1994-09-14 00:00:00,133,9.4,法国
美国,34街的,768,剧情/家庭/奇幻,1994-12-23 00:00:00,114,7.9,美国
中国大陆,活着,202794,剧情/家庭,1994-05-18 00:00:00,132,9.0,法国
...,...,...,...,...,...,...,...
美国,鬼精灵2： 恐怖,60,喜剧/恐怖/奇幻,1994-04-08 00:00:00,85,5.8,美国
英国,黑色第16,44,剧情/惊悚,1996-02-01 00:00:00,106,6.8,美国
日本,蜡笔小新之布里布里王国的秘密宝藏 クレヨンしんちゃん ブリブリ王国の,2142,动画,1994-04-23 00:00:00,94,7.7,日本


In [50]:
df.loc['美国']

,名字,投票人数,类型,上映时间,时长,评分,首映地点
年代,,,,,,,
1957,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.500000,美国
1994,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.400000,洛杉矶首映
2012,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.400000,中国大陆
1993,辛德勒的名单,306904,剧情/历史/战争,1993-11-30 00:00:00,195,9.400000,华盛顿首映
2016,疯狂动物城,284652,喜剧/动作/动画/冒险,2016-03-04 00:00:00,109,9.300000,中国大陆/美国
...,...,...,...,...,...,...,...
1986,离别秋波,240,剧情/爱情/音乐,1986-02-19 00:00:00,90,8.200000,美国
1986,极乐森林,45,纪录片,1986-09-14 00:00:00,90,8.100000,美国
1935,1935年,57,喜剧/歌舞,1935-03-15 00:00:00,98,7.600000,美国


### 取消层次化索引 

In [23]:
df = df.reset_index()
df[:5]

,年代,产地,名字,投票人数,类型,上映时间,时长,评分,首映地点
0,1994,美国,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节
1,1957,美国,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国
2,1997,意大利,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利
3,1994,美国,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
4,1993,中国大陆,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港


### (2)数据旋转 

行列转化：以前5部电影为例

In [51]:
data = df[:5]
data

,,名字,投票人数,类型,上映时间,时长,评分,首映地点
产地,年代,,,,,,,
美国,1957,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国
意大利,1997,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利
美国,1994,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
中国大陆,1993,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港
美国,2012,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆


.T可以直接让数据的行列进行交换

In [52]:
data.T

产地,美国,意大利,美国,中国大陆,美国
年代,1957,1997,1994,1993,2012
名字,控方证人,美丽人生,阿甘正传,霸王别姬,泰坦尼克号
投票人数,42995,327855,580897,478523,157074
类型,剧情/悬疑/犯罪,剧情/喜剧/爱情,剧情/爱情,剧情/爱情/同性,剧情/爱情/灾难
上映时间,1957-12-17 00:00:00,1997-12-20 00:00:00,1994-06-23 00:00:00,1993-01-01 00:00:00,2012-04-10 00:00:00
时长,116,116,142,171,194
评分,9.5,9.5,9.4,9.4,9.4
首映地点,美国,意大利,洛杉矶首映,香港,中国大陆


In [53]:
data

,,名字,投票人数,类型,上映时间,时长,评分,首映地点
产地,年代,,,,,,,
美国,1957,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国
意大利,1997,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利
美国,1994,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
中国大陆,1993,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港
美国,2012,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆


### dataframe也可以使用stack和unstack，转化为层次化索引的Series 

In [26]:
data.stack()

0  年代                     1994
   产地                       美国
   名字                   肖申克的救赎
   投票人数                 692795
   类型                    剧情/犯罪
   上映时间    1994-09-10 00:00:00
   时长                      142
   评分                      9.6
   首映地点                 多伦多电影节
1  年代                     1957
   产地                       美国
   名字                     控方证人
   投票人数                  42995
   类型                 剧情/悬疑/犯罪
   上映时间    1957-12-17 00:00:00
   时长                      116
   评分                      9.5
   首映地点                     美国
2  年代                     1997
   产地                      意大利
   名字                    美丽人生 
   投票人数                 327855
   类型                 剧情/喜剧/爱情
   上映时间    1997-12-20 00:00:00
   时长                      116
   评分                      9.5
   首映地点                    意大利
3  年代                     1994
   产地                       美国
   名字                     阿甘正传
   投票人数                 580897
   类型                    剧情/爱情
   上映时间 

In [27]:
data.stack().unstack()  #转回来

,年代,产地,名字,投票人数,类型,上映时间,时长,评分,首映地点
0,1994,美国,肖申克的救赎,692795,剧情/犯罪,1994-09-10,142,9.6,多伦多电影节
1,1957,美国,控方证人,42995,剧情/悬疑/犯罪,1957-12-17,116,9.5,美国
2,1997,意大利,美丽人生,327855,剧情/喜剧/爱情,1997-12-20,116,9.5,意大利
3,1994,美国,阿甘正传,580897,剧情/爱情,1994-06-23,142,9.4,洛杉矶首映
4,1993,中国大陆,霸王别姬,478523,剧情/爱情/同性,1993-01-01,171,9.4,香港


## 3.2 数据分组，分组运算

### GroupBy技术：实现数据的分组，和分组运算，作用类似于数据透视表 
![test2](p2.jpg)

### 按照电影的产地进行分组 

In [74]:
df

,产地,年代,名字,投票人数,类型,上映时间,时长,评分,首映地点
0,美国,1957,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.500000,美国
1,意大利,1997,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.500000,意大利
2,美国,1994,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.400000,洛杉矶首映
3,中国大陆,1993,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.400000,香港
4,美国,2012,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.400000,中国大陆
...,...,...,...,...,...,...,...,...,...
38728,中国大陆,1986,血溅画屏,95,剧情/悬疑/犯罪/武侠/古装,1905-06-08 00:00:00,91,7.100000,美国
38729,中国大陆,1986,魔窟中的幻想,51,惊悚/恐怖/儿童,1905-06-08 00:00:00,78,8.000000,美国
38730,俄罗斯,1977,列宁格勒围困之星火战役 Блокада: Фильм 2: Ленинградский ме...,32,剧情/战争,1905-05-30 00:00:00,97,6.600000,美国
38731,美国,2018,复仇者联盟3,123456,剧情/科幻,2018-05-04 00:00:00,142,6.935635,美国


In [72]:
df = df.drop(['level_0','index'],axis = 1)
df

In [66]:
df = df.reset_index()
df

ValueError: cannot insert level_0, already exists

In [75]:
group = df.groupby(df["产地"])
group

### 先定义一个分组变量group 

In [76]:
type(group)

pandas.core.groupby.generic.DataFrameGroupBy

### 可以计算分组后各个的统计量 

In [77]:
group.mean() 

,年代,投票人数,时长,评分
产地,,,,
中国台湾,1999.009709,8474.864078,87.257282,7.066667
中国大陆,2004.599684,10898.293793,81.432930,6.064703
中国香港,1991.098177,8164.554348,88.531206,6.474053
丹麦,1999.090909,1993.858586,88.101010,7.245960
俄罗斯,1984.899371,1019.134172,95.918239,7.554507
其他,1998.743750,1590.686979,87.066667,7.237448
加拿大,2002.461964,1915.304288,80.109267,6.733610
印度,2005.974790,3210.843137,121.016807,6.872269
墨西哥,1992.815126,1173.218487,91.840336,7.087395


In [78]:
group.sum()

,年代,投票人数,时长,评分
产地,,,,
中国台湾,1235388,5237466,53925,4367.20000
中国大陆,7621488,41435313,309608,23058.00000
中国香港,5678612,23285309,252491,18464.00000
丹麦,395820,394784,17444,1434.70000
俄罗斯,946797,486127,45753,3603.50000
其他,3837588,3054119,167168,13895.90000
加拿大,1447780,1384765,57919,4868.40000
印度,716133,1146271,43203,2453.40000
墨西哥,237145,139613,10929,843.40000


### 计算每年的平均评分 

In [79]:
df["评分"].groupby(df["年代"]).mean()

年代
1888    7.950000
1890    4.800000
1892    7.500000
1894    6.633333
1895    7.575000
          ...   
2013    6.392604
2014    6.259777
2015    6.141960
2016    5.868217
2018    6.935635
Name: 评分, Length: 127, dtype: float64

### 只会对数值变量进行分组运算 

In [80]:
df["年代"] = df["年代"].astype("str")
df.groupby(df["产地"]).median() #不会再对年代进行求取

,投票人数,时长,评分
产地,,,
中国台湾,487.0,92.0,7.1
中国大陆,501.5,90.0,6.4
中国香港,636.5,92.0,6.5
丹麦,181.5,93.5,7.3
俄罗斯,132.0,93.0,7.7
其他,154.0,90.0,7.4
加拿大,251.0,89.0,6.9
印度,138.0,131.0,7.0
墨西哥,174.0,93.0,7.2


### 我们也可以传入多个分组变量 

In [81]:
df.groupby([df["产地"],df["年代"]]).mean() #根据两个变量进行分组

投票人数          时长        评分
产地   年代                                      
中国台湾 1963    121.000000  113.000000  6.400000
     1965    153.666667  105.000000  6.800000
     1966     51.000000   60.000000  7.900000
     1967   4444.000000  112.000000  8.000000
     1968     89.000000   83.000000  7.400000
...                 ...         ...       ...
韩国   2012   5762.537736  100.669811  6.064151
     2013  10189.036036   96.504505  6.098198
     2014   3776.266667   98.666667  5.650833
     2015   3209.247706  100.266055  5.423853
     2016   1739.850000  106.100000  5.730000

[1585 rows x 3 columns]

### 获得每个地区，每一年的电影的评分的均值 

In [94]:
group = df["评分"].groupby([df["产地"], df["年代"]]) # 层次化索引
means = group.mean()
print(type(means),'\n')
print(means,'\n')
print(means.sort_values(ascending=True),'\n')

<class 'pandas.core.series.Series'> 

产地    年代  
中国台湾  1963    6.400000
      1965    6.800000
      1966    7.900000
      1967    8.000000
      1968    7.400000
                ...   
韩国    2012    6.064151
      2013    6.098198
      2014    5.650833
      2015    5.423853
      2016    5.730000
Name: 评分, Length: 1585, dtype: float64 

产地    年代  
中国大陆  2016    4.712000
美国    1890    4.800000
中国大陆  2014    4.963757
      2015    4.969189
美国    1904    5.200000
                ...   
荷兰    2000    9.200000
德国    1967    9.200000
      1988    9.400000
印度    1983    9.500000
阿根廷   1989    9.600000
Name: 评分, Length: 1585, dtype: float64 



In [37]:
means = group = df["评分"].groupby([df["产地"], df["年代"]]).mean()
means

产地    年代  
中国台湾  1963    6.400000
      1965    6.800000
      1966    7.900000
      1967    8.000000
      1968    7.400000
                ...   
韩国    2012    6.035238
      2013    6.062037
      2014    5.650833
      2015    5.423853
      2016    5.730000
Name: 评分, Length: 1578, dtype: float64

### Series通过unstack方法转化为dataframe

### 会产生缺失值

In [39]:
means.unstack().T

产地,中国台湾,中国大陆,中国香港,丹麦,俄罗斯,其他,加拿大,印度,墨西哥,巴西,...,波兰,泰国,澳大利亚,瑞典,美国,英国,荷兰,西班牙,阿根廷,韩国
年代,,,,,,,,,,,,,,,,,,,,,
1888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.950000,NaN,NaN,NaN,NaN
1890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.800000,NaN,NaN,NaN,NaN,NaN
1892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.450000,NaN,NaN,NaN,NaN,NaN
1895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,7.076471,5.306500,6.105714,6.555556,6.875000,6.853571,6.018182,6.400000,6.983333,8.00,...,6.966667,5.568000,6.76000,7.100,6.308255,7.460140,6.33,6.358333,6.616667,6.062037
2014,6.522222,4.963830,5.616667,7.120000,7.175000,6.596250,5.921739,6.374194,7.250000,6.86,...,7.060000,5.653571,6.56875,6.960,6.393056,7.253398,7.30,6.868750,7.150000,5.650833
2015,6.576000,4.969189,5.589189,7.166667,7.342857,6.732727,6.018750,6.736364,6.500000,6.76,...,6.300000,5.846667,6.88000,7.625,6.231486,7.123256,6.70,6.514286,7.233333,5.423853


## 3.3 离散化处理 

**在实际的数据分析项目中，对有的数据属性，我们往往并不关注数据的绝对取值，只关心它所处的区间或者等级**

**比如，我们可以把评分9分及以上的电影定义为A，7到9分定义为B，5到7分定义为C，3到5分定义为D，小于3分定义为E。**

**离散化也可称为分组、区间化。**

Pandas为我们提供了方便的函数cut():

pd.cut(x,bins,right = True,labels = None, retbins = False,precision = 3,include_lowest = False) 参数解释：

x：需要离散化的数组、Series、DataFrame对象

bins：分组的依据

right = True，

labels：是否要用标记来替换返回出来的数组

retbins：返回x当中每一个值对应的bins的列表

precision：精度。

include_lowest = False

In [97]:
df["评分等级"] = pd.cut(df["评分"], [0,3,5,7,9,10], labels = ['E','D','C','B','A']) #labels要和区间划分一一对应
df

,产地,年代,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级
0,美国,1957,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.500000,美国,A
1,意大利,1997,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.500000,意大利,A
2,美国,1994,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.400000,洛杉矶首映,A
3,中国大陆,1993,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.400000,香港,A
4,美国,2012,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.400000,中国大陆,A
...,...,...,...,...,...,...,...,...,...,...
38728,中国大陆,1986,血溅画屏,95,剧情/悬疑/犯罪/武侠/古装,1905-06-08 00:00:00,91,7.100000,美国,B
38729,中国大陆,1986,魔窟中的幻想,51,惊悚/恐怖/儿童,1905-06-08 00:00:00,78,8.000000,美国,B
38730,俄罗斯,1977,列宁格勒围困之星火战役 Блокада: Фильм 2: Ленинградский ме...,32,剧情/战争,1905-05-30 00:00:00,97,6.600000,美国,C
38731,美国,2018,复仇者联盟3,123456,剧情/科幻,2018-05-04 00:00:00,142,6.935635,美国,C


**同样的，我们可以根据投票人数来刻画电影的热门**

**投票越多的热门程度越高**

In [99]:
bins = np.percentile(df["投票人数"], [0, 20, 40, 60, 80, 100])  # 获取分位数
bins

array([-1.18000e+02,  7.80000e+01,  1.99000e+02,  6.17000e+02,
        2.74660e+03,  6.62552e+05])

In [98]:
df["热门程度"] = pd.cut(df["投票人数"],bins,labels = ['E','D','C','B','A'])
df[:5]

,产地,年代,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
0,美国,1957,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国,A,A
1,意大利,1997,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利,A,A
2,美国,1994,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映,A,A
3,中国大陆,1993,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港,A,A
4,美国,2012,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆,A,A


**大烂片集合：投票人数很多，评分很低**

**遗憾的是，我们可以发现，烂片几乎都是中国大陆的**

In [100]:
df[(df.热门程度 == 'A') & (df.评分等级 == 'E')]

,产地,年代,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
654,中国大陆,2011,B区,5187,剧情/惊悚/恐怖,2011-06-03 00:00:00,89,2.3,中国大陆,E,A
4375,中国大陆,2014,怖偶,4867,悬疑/惊悚,2014-05-07 00:00:00,88,2.8,中国大陆,E,A
5412,中国大陆,2011,床下有人,4309,悬疑/惊悚,2011-10-14 00:00:00,100,2.8,中国大陆,E,A
6801,中国大陆,2013,帝国秘符,4351,动作/冒险,2013-09-18 00:00:00,93,3.0,中国大陆,E,A
8231,中国大陆,2011,飞天,4764,剧情,2011-07-01 00:00:00,115,2.9,中国大陆,E,A
8405,中国大陆,2014,分手达人,3937,喜剧/爱情,2014-06-06 00:00:00,90,2.7,中国大陆,E,A
9600,中国大陆,2012,孤岛惊魂,2982,悬疑/惊悚/恐怖,2013-01-26 00:00:00,93,2.8,中国大陆,E,A
10512,中国大陆,2013,海天盛宴·韦口,3788,情色,2013-10-12 00:00:00,88,2.9,网络,E,A
16798,中国大陆,2013,孪生密码,6390,动作/悬疑,2013-11-08 00:00:00,96,2.9,中国大陆,E,A
21535,日本,2010,拳皇,6329,动作/科幻/冒险,2012-10-12 00:00:00,93,3.0,中国大陆,E,A


### 冷门高分电影

In [101]:
df[(df.热门程度 == 'E') & (df.评分等级 == 'A')]

,产地,年代,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
455,美国,2015,NaN,45,音乐,2015-12-14 00:00:00,60,9.3,美国,A,E
586,英国,2011,BBC喜剧音,38,喜剧/音乐/歌舞,2011-08-13 00:00:00,95,9.3,美国,A,E
641,美国,2003,NaN,38,纪录片/音乐,2003-02-03 00:00:00,55,9.2,美国,A,E
685,英国,2014,NaN,63,音乐/舞台艺术,2014-05-18 00:00:00,49,9.5,美国,A,E
697,英国,2008,NaN,52,纪录片,2008-07-25 00:00:00,40,9.8,美国,A,E
...,...,...,...,...,...,...,...,...,...,...,...
37581,英国,1986,歌唱神探,36,剧情/悬疑/歌舞,1986-11-16 00:00:00,415,9.1,美国,A,E
37707,美国,1972,NaN,56,纪录片,1972-12-23 00:00:00,70,9.3,美国,A,E
38113,美国,1975,山那边,70,剧情,1975-11-14 00:00:00,103,9.1,美国,A,E
38446,美国,2015,奎,62,纪录片/短片,2015-08-19 00:00:00,9,9.1,纽约电影论坛,A,E


### 将处理后的数据进行保存 

In [102]:
df.to_excel("movie_data3.xlsx")

## 3.4 合并数据集 

### （ 1 ）append 
先把数据集拆分为多个，再进行合并

In [50]:
df_usa = df[df.产地 == "美国"]
df_china = df[df.产地 == "中国大陆"]

In [51]:
df_china.append(df_usa) #直接追加到后面，最好是变量相同的

,年代,产地,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
4,1993,中国大陆,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.400000,香港,A,A
21,1961,中国大陆,大闹天宫,74881,动画/奇幻,1905-05-14 00:00:00,114,9.200000,上集,A,A
29,2015,中国大陆,穹顶之下,51113,纪录片,2015-02-28 00:00:00,104,9.200000,中国大陆,A,A
38,1982,中国大陆,茶馆,10678,剧情/历史,1905-06-04 00:00:00,118,9.200000,美国,A,A
45,1988,中国大陆,山水情,10781,动画/短片,1905-06-10 00:00:00,19,9.200000,美国,A,A
...,...,...,...,...,...,...,...,...,...,...,...
38151,1987,美国,零下的激情,199,剧情/爱情/犯罪,1987-11-06 00:00:00,98,7.400000,美国,B,D
38153,1986,美国,离别秋波,240,剧情/爱情/音乐,1986-02-19 00:00:00,90,8.200000,美国,B,C
38156,1986,美国,极乐森林,45,纪录片,1986-09-14 00:00:00,90,8.100000,美国,B,E
38158,1935,美国,1935年,57,喜剧/歌舞,1935-03-15 00:00:00,98,7.600000,美国,B,E


将这两个数据集进行合并

### （ 2 ）merge 

```python
pd.merge(left, right, how = 'inner', on = None, left_on = None, right_on = None,
    left_index = False, right_index = False, sort = True,
    suffixes = ('_x', '_y'), copy = True, indicator = False, validate=None) 
```

left : DataFrame

right : DataFrame or named Series
    Object to merge with.

how : {'left', 'right', 'outer', 'inner'}, default 'inner'
    Type of merge to be performed.

    * left: use only keys from left frame, similar to a SQL left outer join;
      preserve key order.
    * right: use only keys from right frame, similar to a SQL right outer join;
      preserve key order.
    * outer: use union of keys from both frames, similar to a SQL full outer
      join; sort keys lexicographically.
    * inner: use intersection of keys from both frames, similar to a SQL inner
      join; preserve the order of the left keys.
      
on : label or list
    Column or index level names to join on. These must be found in both
    DataFrames. If `on` is None and not merging on indexes then this defaults
    to the intersection of the columns in both DataFrames.
    
left_on : label or list, or array-like
    Column or index level names to join on in the left DataFrame. Can also
    be an array or list of arrays of the length of the left DataFrame.
    These arrays are treated as if they are columns.
    
right_on : label or list, or array-like
    Column or index level names to join on in the right DataFrame. Can also
    be an array or list of arrays of the length of the right DataFrame.
    These arrays are treated as if they are columns.
    
left_index : bool, default False
    Use the index from the left DataFrame as the join key(s). If it is a
    MultiIndex, the number of keys in the other DataFrame (either the index
    or a number of columns) must match the number of levels.
    
right_index : bool, default False
    Use the index from the right DataFrame as the join key. Same caveats as
    left_index.
    
sort : bool, default False
    Sort the join keys lexicographically in the result DataFrame. If False,
    the order of the join keys depends on the join type (how keyword).
    
suffixes : tuple of (str, str), default ('_x', '_y')
    Suffix to apply to overlapping column names in the left and right
    side, respectively. To raise an exception on overlapping columns use
    (False, False).
    
copy : bool, default True
    If False, avoid copy if possible.
    
indicator : bool or str, default False
    If True, adds a column to output DataFrame called "_merge" with
    information on the source of each row.
    If string, column with information on source of each row will be added to
    output DataFrame, and column will be named value of string.
    Information column is Categorical-type and takes on a value of "left_only"
    for observations whose merge key only appears in 'left' DataFrame,
    "right_only" for observations whose merge key only appears in 'right'
    DataFrame, and "both" if the observation's merge key is found in both.

validate : str, optional
    If specified, checks if merge is of specified type.

    * "one_to_one" or "1:1": check if merge keys are unique in both
      left and right datasets.
    * "one_to_many" or "1:m": check if merge keys are unique in left
      dataset.
    * "many_to_one" or "m:1": check if merge keys are unique in right
      dataset.
    * "many_to_many" or "m:m": allowed, but does not result in checks.

    .. versionadded:: 0.21.0

**Returns**

DataFrame
    A DataFrame of the two merged objects.

**See Also**

merge_ordered : Merge with optional filling/interpolation.
merge_asof : Merge on nearest keys.
DataFrame.join : Similar method using indices.

**Notes**

Support for specifying index levels as the `on`, `left_on`, and
`right_on` parameters was added in version 0.23.0
Support for merging named Series objects was added in version 0.24.0

### 我们选取6部热门电影

In [55]:
df1 = df.loc[:5]
df1

,年代,产地,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
0,1994,美国,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节,A,A
1,1957,美国,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国,A,A
2,1997,意大利,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利,A,A
3,1994,美国,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映,A,A
4,1993,中国大陆,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港,A,A
5,2012,美国,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆,A,A


In [56]:
df2 = df.loc[:5][["名字","产地"]]
df2["票房"] = [123344,23454,55556,333,6666,444]

In [57]:
df2

,名字,产地,票房
0,肖申克的救赎,美国,123344
1,控方证人,美国,23454
2,美丽人生,意大利,55556
3,阿甘正传,美国,333
4,霸王别姬,中国大陆,6666
5,泰坦尼克号,美国,444


In [62]:
df2 = df2.sample(frac = 1) #打乱数据

In [63]:
df2.index = range(len(df2))
df2

,名字,产地,票房
0,泰坦尼克号,美国,444
1,阿甘正传,美国,333
2,控方证人,美国,23454
3,美丽人生,意大利,55556
4,霸王别姬,中国大陆,6666
5,肖申克的救赎,美国,123344


现在，我们需要把df1和df2合并

我们发现，df2有票房数据，df1有评分等其他信息  
由于样本的顺序不一致，因此不能直接采取直接复制的方法

In [64]:
pd.merge(df1, df2, how = "inner", on = "名字")

,年代,产地_x,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度,产地_y,票房
0,1994,美国,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节,A,A,美国,123344
1,1957,美国,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国,A,A,美国,23454
2,1997,意大利,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利,A,A,意大利,55556
3,1994,美国,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映,A,A,美国,333
4,1993,中国大陆,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港,A,A,中国大陆,6666
5,2012,美国,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆,A,A,美国,444


由于两个数据集都存在产地，因此合并后会有两个产地信息

### （ 3 ）concat
将多个数据集进行批量合并

In [65]:
df1 = df[:10]
df2 = df[100:110]
df3 = df[200:210]
dff = pd.concat([df1,df2,df3],axis = 0) #默认axis = 0，列拼接需要修改为1
dff

,年代,产地,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
0,1994,美国,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节,A,A
1,1957,美国,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国,A,A
2,1997,意大利,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利,A,A
3,1994,美国,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映,A,A
4,1993,中国大陆,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港,A,A
5,2012,美国,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆,A,A
6,1993,美国,辛德勒的名单,306904,剧情/历史/战争,1993-11-30 00:00:00,195,9.4,华盛顿首映,A,A
7,1997,日本,新世纪福音战士剧场版：Air/真心为你 新世紀エヴァンゲリオン劇場版 Ai,24355,剧情/动作/科幻/动画/奇幻,1997-07-19 00:00:00,87,9.4,日本,A,A
8,2013,日本,银魂完结篇：直到永远的万事屋 劇場版 銀魂 完結篇 万事屋よ,21513,剧情/动画,2013-07-06 00:00:00,110,9.4,日本,A,A
9,1994,法国,这个杀手不太冷,662552,剧情/动作/犯罪,1994-09-14 00:00:00,133,9.4,法国,A,A
